In [ ]:
import os
import subprocess
import json
import tempfile

def get_video_properties(video_path):
    """Get video properties using ffprobe"""
    cmd = [
        "ffprobe",
        "-v", "quiet",
        "-print_format", "json",
        "-show_streams",
        "-show_format",
        video_path
    ]
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        data = json.loads(result.stdout)

        # Get duration from format section (more reliable)
        duration = float(data.get("format", {}).get("duration", 0))

        for stream in data.get("streams", []):
            if stream.get("codec_type") == "video":
                return {
                    "width": int(stream.get("width", 0)),
                    "height": int(stream.get("height", 0)),
                    "codec_name": stream.get("codec_name", ""),
                    "frame_rate": eval(stream.get("r_frame_rate", "0/1")),
                    "duration": duration
                }
        return None
    except (subprocess.CalledProcessError, json.JSONDecodeError) as e:
        print(f"Error getting video properties for {video_path}: {e}")
        return None

def crop_and_trim_video(input_path, output_path, target_width=1080, target_height=1920, target_duration=6):
    """
    Crops video to target dimensions suitable for shorts/reels (9:16 aspect ratio)
    and trims it to the target duration in seconds.
    """
    # Get original dimensions and duration
    props = get_video_properties(input_path)
    if not props:
        return False

    orig_width = props["width"]
    orig_height = props["height"]
    orig_duration = props["duration"]

    # Calculate crop dimensions to maintain aspect ratio
    target_aspect = target_width / target_height  # 9:16 = 0.5625
    orig_aspect = orig_width / orig_height

    if orig_aspect > target_aspect:
        # Video is wider than target aspect ratio, crop from sides
        new_width = int(orig_height * target_aspect)
        crop_x = (orig_width - new_width) // 2
        crop_y = 0
        crop_width = new_width
        crop_height = orig_height
    else:
        # Video is taller than target aspect ratio, crop from top and bottom
        new_height = int(orig_width / target_aspect)
        crop_x = 0
        crop_y = (orig_height - new_height) // 2
        crop_width = orig_width
        crop_height = new_height

    # Calculate speed factor to fit target duration
    # If orig_duration is 12 seconds and target is 6, speed_factor will be 2.0 (2x faster)
    speed_factor = orig_duration / target_duration

    # Create ffmpeg filter for cropping, scaling, and adjusting speed
    filter_complex = f"crop={crop_width}:{crop_height}:{crop_x}:{crop_y},scale={target_width}:{target_height}"

    # Add speed adjustment if necessary (with a maximum reasonable limit)
    if speed_factor > 1.0:
        if speed_factor > 10.0:
            print(f"Warning: Video {input_path} needs to be sped up by {speed_factor}x which is very fast. Limiting to 10x.")
            speed_factor = 10.0
        filter_complex += f",setpts=PTS/{speed_factor}"

    # Create ffmpeg command for processing
    cmd = [
        "ffmpeg",
        "-i", input_path,
        "-vf", filter_complex,
        "-c:v", "libx264",
        "-preset", "ultrafast",
        "-crf", "18",
        "-an",  # No audio
        "-t", str(target_duration),  # Set max duration
        "-y",
        output_path
    ]

    try:
        subprocess.run(cmd, capture_output=True, text=True, check=True)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Error processing video {input_path}: {e.stderr}")
        return False

def merge_videos_with_equal_duration(base_folder, total_duration=30, shorts_width=1080, shorts_height=1920):
    """
    Processes videos to equal duration segments totaling the specified duration,
    then concatenates them without further re-encoding.
    """
    # Find all subdirectories that contain video files
    for root, dirs, files in os.walk(base_folder):
        # Filter out mp4 files
        video_files = [f for f in files if f.lower().endswith('.mp4')]
        if not video_files:
            continue  # Skip folders with no video files

        # Skip the merged video if it exists
        video_files = [f for f in video_files if f != 'merged_video.mp4']
        if not video_files:
            continue

        # Get full paths of video files
        video_paths = [os.path.join(root, f) for f in video_files]

        # Sort video files to maintain consistent order
        video_paths.sort()

        num_videos = len(video_paths)
        print(f"Found {num_videos} videos in '{root}'")

        if num_videos < 2:
            print(f"Need at least 2 videos to merge in '{root}'. Skipping.")
            continue

        # Calculate target duration for each video segment
        segment_duration = total_duration / num_videos
        print(f"Each video will be trimmed to {segment_duration:.2f} seconds to fit within {total_duration} seconds total")

        # Create a temporary directory for processed videos
        with tempfile.TemporaryDirectory() as temp_dir:
            processed_paths = []

            # Process all videos to equal duration with shorts format
            for i, video_path in enumerate(video_paths):
                # Process the video
                processed_path = os.path.join(temp_dir, f"processed_{i}.mp4")
                print(f"Processing video {i+1}/{num_videos} to {segment_duration:.2f} seconds at {shorts_width}x{shorts_height}")

                if crop_and_trim_video(video_path, processed_path, shorts_width, shorts_height, segment_duration):
                    processed_paths.append(processed_path)
                else:
                    print(f"Failed to process video {video_path}, skipping")
                    continue

            if len(processed_paths) < 2:
                print(f"Not enough videos were successfully processed in '{root}'. Skipping merging.")
                continue

            # Create a file list for the concat demuxer
            list_file_path = os.path.join(temp_dir, 'file_list.txt')
            with open(list_file_path, 'w', encoding='utf-8') as list_file:
                for path in processed_paths:
                    # Make sure to escape single quotes in filenames
                    escaped_path = path.replace("'", "'\\''")
                    list_file.write(f"file '{escaped_path}'\n")

            # Use concat demuxer to combine all processed videos
            output_file = os.path.join(root, 'merged_video.mp4')
            concat_cmd = [
                "ffmpeg",
                "-f", "concat",
                "-safe", "0",
                "-i", list_file_path,
                "-c", "copy",  # Just copy streams, no re-encoding
                "-an",  # No audio
                "-y",  # Overwrite output if exists
                output_file
            ]

            print(f"Concatenating videos in '{root}'...")
            try:
                subprocess.run(concat_cmd, capture_output=True, text=True, check=True)
                # Verify final duration
                final_props = get_video_properties(output_file)
                if final_props:
                    print(f"Successfully merged videos in '{root}' into '{output_file}'")
                    print(f"Final video duration: {final_props['duration']:.2f} seconds")
                else:
                    print(f"Successfully merged videos but couldn't verify final duration")
            except subprocess.CalledProcessError as e:
                print(f"Error concatenating videos in '{root}': {e.stderr}")

if __name__ == "__main__":
    base_folder = os.path.join("Storage", "downloaded_videos")

    # Process 5 videos to fit in 30 seconds total (6 seconds each)
    # Standard shorts/reels aspect ratio is 9:16 (1080x1920 is common)
    merge_videos_with_equal_duration(base_folder, total_duration=30, shorts_width=1080, shorts_height=1920)